In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup
import requests


In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')

#print(soup.prettify())
#soup.title.string
#soup.find_all("tr")


In [3]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
Toronto_nbh = pd.DataFrame(columns=column_names)

In [4]:
for data in soup.find_all('tr'):
    data_1 = data.find('td')
#    print(data_1)
    if data_1 != None and data_1.string != None:
        if data_1.string[0] == 'M':
            postalcode = data_1.string
#            print(postalcode)
            
            data_2 = data_1.next_sibling.next_sibling
#            print(data_2)
            try:
                borough = data_2.find('a').text
#                print(borough)

                data_3 = data_2.next_sibling.next_sibling
#                print(data_3)
                try:
                    neighborhood = data_3.find('a').text
                except Exception as e:
                    neighborhood = borough  #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
#                print(neighborhood)
            
                Toronto_nbh = Toronto_nbh.append({'PostalCode': postalcode,
                                                  'Borough': borough,
                                                  'Neighborhood': neighborhood}, ignore_index=True)
            
            except Exception as e:
                borough = data_2.string  #Ignore cells with a borough that is Not assigned.
                            


In [5]:
Toronto_nbh.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [6]:
def nstrcat(arr):
    return ','.join(arr)

Toronto_nbh = Toronto_nbh.groupby(['PostalCode','Borough'], as_index=False).agg(nstrcat)

#Toronto_nbh = Toronto_nbh.groupby(['PostalCode','Borough'], as_index=False).str.cat(sep=',')

In [7]:
Toronto_nbh.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Scarborough,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Scarborough
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Scarborough,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough"
9,M1N,Scarborough,"Birch Cliff,Scarborough"


In [8]:
Toronto_nbh.shape

(100, 3)

In [9]:
#Get the geographical coordinates of each postal code
df_coords = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
df_coords.shape

(103, 3)

In [10]:
#Add the coordinates into the Toronto_nbh dataframe
Toronto_nbh = Toronto_nbh.join(df_coords.set_index('Postal Code'), on='PostalCode')
Toronto_nbh.head(100)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Scarborough,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Scarborough",43.692657,-79.264848


In [11]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [12]:
Toronto_data = Toronto_nbh[Toronto_nbh['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head(100)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"East Toronto,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"East Toronto,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,East Toronto,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Central Toronto,43.712751,-79.390197
6,M4R,Central Toronto,Central Toronto,43.715383,-79.405678
7,M4S,Central Toronto,Central Toronto,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Central Toronto",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Central Toronto,Rathnelly,South Hill...",43.686412,-79.400049


In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = '4T3VZBFQEC0XGOZ0FIFSO4D2KBCMOE0OJ3NODZI4TQT32M2R' # your Foursquare ID
CLIENT_SECRET = 'HUDZRZPTU5TJPC20DKPAD5QFL4DQFEXYD4ZEUXP4IY3KVIMF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4T3VZBFQEC0XGOZ0FIFSO4D2KBCMOE0OJ3NODZI4TQT32M2R
CLIENT_SECRET:HUDZRZPTU5TJPC20DKPAD5QFL4DQFEXYD4ZEUXP4IY3KVIMF


In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
East Toronto,Riverdale
East Toronto,India Bazaar
East Toronto
Lawrence Park
Central Toronto
Central Toronto
Central Toronto
Moore Park,Central Toronto
Deer Park,Central Toronto,Rathnelly,South Hill,Central Toronto
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Downtown Toronto,Downtown Toronto
St. James Town
Berczy Park
Downtown Toronto
Downtown Toronto,Downtown Toronto,Downtown Toronto
Downtown Toronto,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Downtown Toronto
Central Toronto
Forest Hill North,Central Toronto
The Annex,Central Toronto,Yorkville
Downtown Toronto,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,King and Spadina,Railway Lands,South Niagara
Downtown Toronto
First Canadian Place,Underground city
Downtown Toronto
Dovercourt Village,West Toronto
Little Portugal,Trinity
West Toronto,Exhibition Place,Parkdale Village

In [20]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1709, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"East Toronto,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [21]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower,Downtown Toronto,Downtown Toronto,Downtown Toronto,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Toronto,59,59,59,59,59,59
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Church and Wellesley,86,86,86,86,86,86
"Commerce Court,Downtown Toronto",100,100,100,100,100,100
"Deer Park,Central Toronto,Rathnelly,South Hill,Central Toronto",15,15,15,15,15,15
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


In [23]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
Toronto_onehot.shape

(1709, 233)

In [25]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033898,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.00000,0.00,0.060000,0.000000,0.050000,0.010000,0.000000
5,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.011628,0.000000,0.000000,0.00000,0.00,0.000000,0.011628,0.011628,0.000000,0.011628
6,"Commerce Court,Downtown Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,"Deer Park,Central Toronto,Rathnelly,South Hill...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000
8,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.000000,0.00000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Dovercourt Village,West Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Park
1,"CN Tower,Downtown Toronto,Downtown Toronto,Dow...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Bar,Harbor / Marina
2,"Cabbagetown,St. James Town",Coffee Shop,Café,Market,Restaurant,Bakery,Pub,Pizza Place,Park,Italian Restaurant,Jewelry Store
3,Central Toronto,Sandwich Place,Dessert Shop,Coffee Shop,Park,Clothing Store,Toy / Game Store,Italian Restaurant,Gym,Café,Diner
4,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Bar,Mexican Restaurant,Dumpling Restaurant,Bakery,Chinese Restaurant,Coffee Shop,Gaming Cafe


In [28]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"East Toronto,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"East Toronto,India Bazaar",43.668999,-79.315572,0,Park,Pizza Place,Liquor Store,Fast Food Restaurant,Italian Restaurant,Food & Drink Shop,Sushi Restaurant,Brewery,Ice Cream Shop,Fish & Chips Shop
3,M4M,East Toronto,East Toronto,43.659526,-79.340923,0,Café,Coffee Shop,Yoga Studio,American Restaurant,Gastropub,Gym / Fitness Center,Italian Restaurant,Light Rail Station,Brewery,Bakery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Lake,Swim School,Dim Sum Restaurant,Bus Line,Wings Joint,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,East Toronto,0,Park,Pizza Place,Liquor Store,Fast Food Restaurant,Italian Restaurant,Food & Drink Shop,Sushi Restaurant,Brewery,Ice Cream Shop,Fish & Chips Shop
3,East Toronto,0,Café,Coffee Shop,Yoga Studio,American Restaurant,Gastropub,Gym / Fitness Center,Italian Restaurant,Light Rail Station,Brewery,Bakery
5,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Park,Clothing Store,Toy / Game Store,Italian Restaurant,Gym,Café,Diner
6,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Park,Clothing Store,Toy / Game Store,Italian Restaurant,Gym,Café,Diner
7,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Park,Clothing Store,Toy / Game Store,Italian Restaurant,Gym,Café,Diner
9,Central Toronto,0,Coffee Shop,Pub,Bagel Shop,Sports Bar,Vietnamese Restaurant,Liquor Store,Restaurant,Supermarket,Sushi Restaurant,Pizza Place
11,Downtown Toronto,0,Coffee Shop,Café,Market,Restaurant,Bakery,Pub,Pizza Place,Park,Italian Restaurant,Jewelry Store
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Men's Store,Gastropub,Hotel,Fast Food Restaurant,Gym
13,Downtown Toronto,0,Coffee Shop,Bakery,Park,Mexican Restaurant,Gym / Fitness Center,Breakfast Spot,Café,Restaurant,Pub,Theater


In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Park,Lake,Swim School,Dim Sum Restaurant,Bus Line,Wings Joint,Discount Store,Event Space,Ethiopian Restaurant,Electronics Store


In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Park,Playground,Summer Camp,Trail,Restaurant,Comic Shop,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Downtown Toronto,3,Park,Playground,Trail,Building,Wings Joint,Dim Sum Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
